# Lösung Übung 2 - Woche 3 und 4 - Transfer Learning

Hier findet ihr eine Lösung, die zu einem optimalen Ergebnis für die Übung 2 aus Woche 3 und 4 führt.

Dafür wird Transfer Learning genutzt. Da der Datensatz ImageWoof Teilmenge des Imagenet Datensatzes ist, auf dem das Netz schon trainiert ist, erhält man sehr schnell sehr gute Ergebnisse.

In [0]:
# magic command for Colab Notebooks
%tensorflow_version 2.x

In [0]:
# TensorFlow ≥2.0 is required for this notebook
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# check if GPU is available as this notebook will be very slow without GPU
if not tf.test.is_gpu_available():
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")


In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.models import Model

# jupyters magic command
%matplotlib inline

In [0]:
# resize the images to a uniform size
def preprocess(image, label):
    resized_image = tf.image.resize(image, [300, 300]) 
    # run Xceptions preprocessing function
    # preprocessed_image = tf.keras.applications.xception.preprocess_input(resized_image)
    return resized_image, label

## Loading and Preprocessing

In [0]:
!pip install --upgrade deeplearning2020

from deeplearning2020.datasets import ImageWoof

train_data, test_data, classes = ImageWoof.load_data()

     |████████████████████████████████| 399kB 3.5MB/s 
  Created wheel for deeplearning2020: filename=deeplearning2020-0.4.21-py2.py3-none-any.whl size=8548 sha256=8b767a9d92fc7ab88de95eb4ed4b22c139f707e1aaf712a4ef9451e7ed46ad06
  Stored in directory: /root/.cache/pip/wheels/7f/c2/8a/f9f03fc839999f1fe9d5e5a9d2c97cdd5cb8329f61f82ea2c9
  Created wheel for kerasltisubmission: filename=kerasltisubmission-0.4.9-py2.py3-none-any.whl size=8867 sha256=5591ebedad69abc9c44b23f22f7fdb96efbd789b7c5d27c6fbff015c3088a960
  Stored in directory: /root/.cache/pip/wheels/fd/61/f7/09171376b25408ae21b58e98c9fbf2eb924f676bb77659f983
Successfully built deeplearning2020 kerasltisubmission
328294400/328288506 [==============================] - 6s 0us/step
/root/.keras/datasets/imagewoof2-320/train
Loaded 9025 images
/root/.keras/datasets/imagewoof2-320/val
Loaded 3929 images


In [0]:
train_data = train_data.map(preprocess).batch(32).prefetch(1)

In [0]:
test_data = test_data.map(preprocess).batch(32).prefetch(1)

In [0]:
n_classes = len(classes)
print(classes)

['Shih-Tzu' 'Rhodesian ridgeback' 'Beagle' 'English foxhound'
 'Border terrier' 'Australian terrier' 'Golden retriever'
 'Old English sheepdog' 'Samoyed' 'Dingo']


## Definition and Training

In [0]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D


# build a transfer learning model with Xception and a new Fully-Connected-Classifier
base_model = Xception(
    weights='imagenet',
    include_top=False
)
model = GlobalAveragePooling2D()(base_model.output)
model = Dropout(0.5)(model)

# include new Fully-Connected-Classifier
output_layer = Dense(n_classes, activation='softmax')(model)

# create Model
model = Model(base_model.input, output_layer)

In [0]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [0]:
# set the pretrained layers to not trainable because
# there are already trained and we don't want to destroy
# their weights
for layer in base_model.layers:
  layer.trainable = False

In [0]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    train_data,
    epochs=2,
    validation_data=test_data
)

Epoch 1/2
283/283 [==============================] - 54s 189ms/step - loss: 0.1774 - accuracy: 0.9543 - val_loss: 0.1251 - val_accuracy: 0.9677
Epoch 2/2
283/283 [==============================] - 53s 186ms/step - loss: 0.0869 - accuracy: 0.9753 - val_loss: 0.1175 - val_accuracy: 0.9695
